In [1]:
import numpy as np
import plotly.express as px
from dubious import Normal, LogNormal, Uniform, Uncertain, sample_uncertain
rng = np.random.Generator(np.random.PCG64())

In [4]:
normal = Normal(0, 1)
normal2 = Normal(10, 1)
normal3 = Normal(normal2, normal)


In [5]:
data = normal3.sample(2000, rng)

fig = px.histogram(
    data,
    nbins=50,          # number of bins
    title="My Histogram"
)

fig.show()

C:\Users\Flynn Wilson\Documents\Python Projects\dubious\src\dubious\distributions.py:63: UserWarning:




In [ ]:
mu = Normal(0, 1)
X = Normal(mu, 2)

mu.sample(1000, rng)

array([-0.51067404, -1.09069946,  0.46546311,  0.6564478 ,  1.03170653,
       -1.14704847, -0.16418378,  2.83669029, -0.81245272,  1.27206971,
        0.45171645,  0.13659652, -3.12596477,  0.13656201,  1.93486542,
        0.61907047, -1.18738371, -1.38379946, -0.9407638 ,  0.99604849,
        0.28432   , -0.28512295, -0.41465631,  0.51571165, -0.27796902,
       -0.64873324,  2.07969887, -1.25812091, -0.11568837, -1.74053326,
       -1.13775129, -0.32681257,  0.24521544,  0.07790931,  0.27351544,
        1.85841473, -0.43374115,  0.13303817, -0.6898251 ,  0.27447967,
       -1.60546304, -0.378111  ,  0.31012382, -1.71261834, -0.82931872,
       -0.76956307,  0.58845759,  1.01046021,  0.93076788,  1.54453044,
       -1.48713266,  0.76406863,  0.78477379, -0.46410453, -0.82054424,
       -1.74575117,  0.42879406, -1.01462884,  0.30399929, -0.93679387,
        0.46582152, -0.29867052,  0.94452954,  1.23726559, -0.88129988,
        0.43170539, -0.22949178,  0.76748547,  1.86501735, -0.39